In [12]:
%matplotlib inline

In [13]:
# Author: Yulou Zhou

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import csv
import glob
import os
import pandas as pd
import numpy as np
import csv
from random import shuffle
from MyDataset import MyDataset
torch.manual_seed(1)

In [24]:
batch_size = 256
train_data = MyDataset('parsedtrain', prevchord = 1)
data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
c=0
for i in data_loader:
    print("p: ", i['p'])
    print("m: ", i['m'])
    print("mask: ", i['mask'])
    print("l: ", i['l'])
    c += 10
    if c > 0: break

p:  tensor([[  1,   5,   8],
        [  4,   8,  11],
        [  1,   5,   8],
        [  8,  12,   3],
        [  3,   7,  10],
        [  6,  10,   1],
        [ 12,   4,   7],
        [  8,  12,   3],
        [  6,  10,   1],
        [  3,   7,  10],
        [ 10,   2,   5],
        [  8,  12,   3],
        [  6,  10,   1],
        [  0,   4,   7],
        [  8,  12,   3],
        [  0,   0,   0],
        [  7,  11,   2],
        [  4,   8,  11],
        [ 11,   3,   6],
        [  6,  10,   1],
        [ 10,   2,   5],
        [ 10,   2,   5],
        [  1,   5,   8],
        [  2,   6,   9],
        [  0,   0,   0],
        [  1,   5,   8],
        [ 12,   4,   7],
        [  5,   9,  12],
        [  1,   5,   8],
        [  1,   5,   8],
        [  6,  10,   1],
        [  3,   7,  10],
        [  7,  11,   2],
        [  4,   8,  11],
        [  3,   7,  10],
        [  1,   5,   8],
        [  3,   7,  10],
        [  3,   7,  10],
        [  3,   7,  10],
        [  5,   9,  1

In [25]:
P_EMBEDDING_DIM = 6
M_EMBEDDING_DIM = 6
P_HIDDEN_DIM = 40
M_HIDDEN_DIM = 20
vocab_size = 14
tagset_size = 13

Create the model:



In [26]:
class LSTMTagger(nn.Module):

    def __init__(self, p_embed_dim, m_embed_dim, p_hidden_dim, m_hidden_dim, vocab_size, tagset_size, batch_size = 1):
        super(LSTMTagger, self).__init__()
        self.batch_size = batch_size
        
        self.p_embed_dim, self.m_embed_dim = p_embed_dim, m_embed_dim
        self.p_embed = nn.Embedding(vocab_size, p_embed_dim)
        self.m_embed = nn.Embedding(vocab_size, m_embed_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.p_lstm = nn.LSTM(p_embed_dim, p_hidden_dim)
        self.m_lstm = nn.LSTM(m_embed_dim, m_hidden_dim)
        

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(p_hidden_dim + m_hidden_dim, tagset_size)
        self.root2second = nn.Linear(p_hidden_dim + m_hidden_dim + 1, tagset_size)
        self.root2third = nn.Linear(p_hidden_dim + m_hidden_dim + 1, tagset_size)
        
        self.p_hidden = self.init_hidden(p_hidden_dim)
        self.m_hidden = self.init_hidden(m_hidden_dim)

    def init_hidden(self, hidden_dim):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, self.batch_size, hidden_dim),
                torch.zeros(1, self.batch_size, hidden_dim))

    def forward(self, prev, melody):
        
        prev_embeds = self.p_embed(prev)
        x1 = prev_embeds.view(-1, self.batch_size, self.p_embed_dim) #TODO: DEBUG THIS LINE (probably the last batch)
        
        #TODO: ensure that melody input is a tensor
        #print("prev", prev_embeds.size())
        #print("melody", melody)
        #melody = torch.LongTensor(list(melody))
        
        melody_embeds =  self.m_embed(melody) #Error here
        x2 = melody_embeds.view(-1, self.batch_size, self.m_embed_dim)
        
        p_lstm_out, self.p_hidden = self.p_lstm(x1, self.p_hidden)
        m_lstm_out, self.m_hidden = self.m_lstm(x2, self.m_hidden)
        #print(p_lstm_out.size(), m_lstm_out.size())
        p_fstate, m_fstate = p_lstm_out[-1], m_lstm_out[-1]
        
        #print("sizes:", prev.view(1, -1).size(), final_state.size())
        
        #TODO: concatenate with extra information
        concat = torch.cat((p_fstate, m_fstate), 1)
        #TODO: increase output dimensions (How?)
        tag_space = self.hidden2tag(concat) #REASON: you only need the final state
        tag_scores = F.log_softmax(tag_space, dim = 1)
        
        #TODO: These are the two other notes
        #print(concat.size(), tag_scores.max(1)[1].float().size())
        withroot = torch.cat((concat, tag_scores.max(1)[1].float().view(-1,1)), 1)
        second = self.root2second(withroot)
        third = self.root2third(withroot)
        second_scores = F.log_softmax(second, dim=1)
        third_scores = F.log_softmax(third, dim=1)
        #print(tag_scores.size(), second_scores.size(), third_scores.size())
        stacked = torch.stack([tag_scores, second_scores, third_scores], dim=0)
        #print(stacked.size())
        return [tag_scores, second_scores, third_scores]

In [27]:
model = LSTMTagger(P_EMBEDDING_DIM, M_EMBEDDING_DIM, P_HIDDEN_DIM, M_HIDDEN_DIM, vocab_size, tagset_size, batch_size = batch_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.2)

Train the model:



In [28]:
with torch.no_grad():
    for i in data_loader:
        p, m, l = i['p'], i['m'], i['l']
        tag_scores = model(p, m)
        print(tag_scores)
        break
    

[tensor([[-2.4946, -2.6821, -2.5503,  ..., -2.4058, -2.7968, -2.3738],
        [-2.5681, -2.6337, -2.5189,  ..., -2.4626, -2.7028, -2.4311],
        [-2.5483, -2.6563, -2.5413,  ..., -2.4467, -2.6928, -2.3946],
        ...,
        [-2.5188, -2.6669, -2.5489,  ..., -2.4137, -2.7567, -2.3791],
        [-2.5588, -2.6789, -2.4798,  ..., -2.4501, -2.6824, -2.4134],
        [-2.5427, -2.6752, -2.5407,  ..., -2.4514, -2.7316, -2.4178]]), tensor([[-0.9867, -3.5238, -2.2445,  ..., -3.3311, -3.0656, -2.3877],
        [-0.9547, -3.6364, -2.2476,  ..., -3.3577, -3.1016, -2.3291],
        [-0.9822, -3.5786, -2.2655,  ..., -3.3235, -3.0604, -2.3571],
        ...,
        [-0.9711, -3.5665, -2.2587,  ..., -3.3313, -3.0832, -2.3835],
        [-0.9836, -3.5769, -2.2230,  ..., -3.3360, -3.1139, -2.3031],
        [-0.9506, -3.6088, -2.2156,  ..., -3.4013, -3.1256, -2.3286]]), tensor([[-2.4891, -3.9646, -2.4469,  ..., -1.9348, -2.3435, -3.8513],
        [-2.5122, -3.9582, -2.5556,  ..., -1.8610, -2.3533,

In [42]:
for epoch in range(500):  # again, normally you would NOT do 300 epochs, it is toy data
    for i in data_loader:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM,
        # detaching it from its history on the last instance.
        model.p_hidden = model.init_hidden(P_HIDDEN_DIM)
        model.m_hidden = model.init_hidden(M_HIDDEN_DIM)

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        p, m, l = i['p'], i['m'], i['l']
        #print(m)
        tag_scores = model(p, m)
        # Step 3. Run our forward pass.

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        #print(targets[:1])
        loss = 0
        for i in range(len(tag_scores)):
            loss += 1 / ((i+1)**2) * loss_function(tag_scores[i], l.view(-1, batch_size)[i])   
        #loss = loss_function(tag_scores, l.view(-1, batch_size))
        loss.backward()
        optimizer.step()
        break
    if(epoch % 10==0): print(epoch, loss)


# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     _, point2 = tag_scores.max(1)
#     print(point2)

0 tensor(2.5639)
10 tensor(2.6934)
20 tensor(2.6348)
30 tensor(2.9000)
40 tensor(2.6886)
50 tensor(2.6760)
60 tensor(2.5877)
70 tensor(2.6720)
80 tensor(2.7008)
90 tensor(2.7404)
100 tensor(2.6998)
110 tensor(2.5505)
120 tensor(2.9395)
130 tensor(2.7135)
140 tensor(2.6105)
150 tensor(2.5302)
160 tensor(2.8258)
170 tensor(2.6571)
180 tensor(2.7375)
190 tensor(2.8081)
200 tensor(2.7389)
210 tensor(2.6739)
220 tensor(2.6422)
230 tensor(2.6666)
240 tensor(2.7190)
250 tensor(2.6424)
260 tensor(2.5507)
270 tensor(2.4947)
280 tensor(2.6155)
290 tensor(2.6298)
300 tensor(2.5811)
310 tensor(2.6262)
320 tensor(2.7342)
330 tensor(2.5940)
340 tensor(2.6780)
350 tensor(2.6042)
360 tensor(2.6819)
370 tensor(2.5078)
380 tensor(2.6201)
390 tensor(2.7426)
400 tensor(2.7676)
410 tensor(2.6746)
420 tensor(2.6647)
430 tensor(2.5857)
440 tensor(2.6986)
450 tensor(2.7160)
460 tensor(2.5578)
470 tensor(2.6317)
480 tensor(2.7005)
490 tensor(2.8715)


In [44]:
accuracy = 0.0
c = 0
for j in range(100):
    for i in data_loader:
        c+=1
        p, m, l = i['p'], i['m'], i['l']
        tag_scores = model(p, m)
        pred = [a.max(1)[1] for a in tag_scores]
        
        truth = l.view(3, -1)[0]
        predicted = pred[0]
        #print(truth, predicted)
        accuracy += np.average((truth == predicted))
        break
print(accuracy/c)
        
#     shuffle(training_data)
#     with torch.no_grad():
#         inputs = prepare_sequence(training_data[0][1], word_to_ix)
#         prevsa = prepare_sequence(training_data[0][0], word_to_ix)
#         prevt = prepare_sequence(['1', '3', '5'], word_to_ix)
#         tag_scores = model(inputs, prevt)

#         # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#         # for word i. The predicted tag is the maximum scoring tag.
#         # Here, we can see the predicted sequence below is 0 1 2 0 1
#         # since 0 is index of the maximum value of row 1,
#         # 1 is the index of maximum value of row 2, etc.
#         # Which is DET NOUN VERB DET NOUN, the correct sequence!
#         _, point2 = tag_scores.max(1)
#         #print(point2.tolist())

#     values, indices = tag_scores.max(1)
#     truth = np.array([int(a) for a in training_data[0][1]])[-1:]
#     predicted = point2.numpy()
#     accuracy += np.average((truth == predicted))
#     c += 1
# print(accuracy/c)

0.0909375
